In [1]:
import os
import traceback
import sys
import code
import pdb
from importlib import reload

import numpy as np
import scipy as sc
import scipy.io as scio
import yaml
import pandas as pd

sys.path.insert(0, '/Users/chris/Workspace/datajoint-python')
os.chdir('/Users/chris/Workspace/djwip')

import datajoint as dj
from datajoint.base_relation import lookup_class_name

dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'chris'
dj.config['database.password'] = ''
dj.config['display.limit'] = 5

In [2]:
schema = dj.schema('tutorial_schemax', locals())

Connecting chris@localhost:3306


In [3]:


@schema
class Animal(dj.Manual):
    definition = '''
    animal_id:		int		# animal id
    ---
    animal_name:	varchar(20)	# animal name
    animal_desc:	varchar(255)	# animal desc
    '''


@schema
class Session(dj.Manual):
    definition = '''
    -> Animal
    session_id:		int		# session id
    ---
    session_date:	date		# session date
    '''


@schema
class TrialType(dj.Lookup):
    definition = '''
    trial_type:		varchar(16)	# trial type
    '''
    contents = zip(['fun', 'games', 'injury'])


@schema
class Trial(dj.Manual):
    definition = '''
    -> Session
    trial_id:		int		# trial id
    ---
    -> TrialType
    trial_result:	bool		# trial result
    '''

In [4]:
Trial()

animal_id animal id,session_id session id,trial_id trial id,trial_type trial type,trial_result trial result
0,0,0,fun,1


In [5]:
from datajoint.base_relation import lookup_class_name

In [6]:
def schema_prep(schema):
    ''' not 100% if needed.. zap if needed after restart '''
    schema.connection.dependencies.load()


def table_to_classname(tablename, schema):
    ''' lookup_class_name wrapper which (will handle) aliases '''
    if not tablename.isdigit():
        return lookup_class_name(tablename, schema.context) or tablename
    else:
        raise NotImplementedError('aliased table_to_class')


def table_to_class(tablename, schema):
    ''' return class instance for a given tablename/schema '''
    tname = table_to_classname(tablename, schema)
    return schema.context[tname]


def schema_iterator(schema):
    return (table_to_class(c, schema)
            for c in schema.connection.dependencies.nodes())


def parent_classes(tableclass, schema):
    ''' return parent classes for a given tableclass/schema '''
    return (table_to_class(c, schema) for c in tableclass().parents())


def child_classes(tableclass, schema):
    ''' return child classes for a given tableclass/schema '''
    return (table_to_class(c, schema) for c in tableclass().children())


def get_table_heading(table):
    ''' get the table heading for a given tableclass '''
    table._heading.init_from_database(
        table.connection, table.database, table.table_name)
    return table._heading


def get_table_attributes(table):
    return get_table_heading(table).attributes

In [10]:
def hacktry(table):
    columns = [k for k in get_table_attributes(table)]
    data = table().fetch()
    return pd.DataFrame(
        data=data,  # TODO: limit or paginate somehow
        columns=columns,
    )


In [11]:
bear = hacktry(Session)
bear

,animal_id,session_id,session_date
0,0,0,2017-10-16


In [12]:
from qgrid import QgridWidget

In [13]:
qw = QgridWidget(df=bear, show_toolbar=True)

In [14]:
qw

A Jupyter Widget

In [15]:
cub = qw.get_changed_df()

In [16]:
cub

,animal_id,session_id,session_date
0,0,0,2017-10-16
1,0,1,2017-10-16


In [44]:
# hmm
cub.isin(bear)

,animal_id,session_id,session_date
0,True,True,True
1,False,False,False


In [46]:
# via https://stackoverflow.com/questions/36891977/pandas-diff-of-two-dataframes
# which also provides a dataframe 'merge' methodology

In [54]:
list(set(cub.index) - set(bear.index))

[1]

In [88]:
cub.iloc[1]

animal_id                0
session_id               1
session_date    2017-10-16
Name: 1, dtype: object

In [87]:
cub[cub.index.isin(list(set(cub.index) - set(bear.index)))]

,animal_id,session_id,session_date
1,0,1,2017-10-16


In [89]:
# now, find a way to get the round trip working somewhat transparently via some convenience object gizmo

In [90]:
updates = cub[cub.index.isin(list(set(cub.index) - set(bear.index)))]

In [91]:
type(updates)

pandas.core.frame.DataFrame